# Reeb Graph Matching
#### Implementing 'Topology Matching for Fully Automatic Similarity Estimation of 3D Shapes'
#### http://graphics.stanford.edu/courses/cs468-08-fall/pdf/Hilaga01.pdf
#### (Hilaga et al., 2001)

In [1]:
import numpy as np
import os
from os.path import isfile, join
from os import listdir
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import csd_functions
import scipy
import networkx as nx
sns.set()


Load tree data

In [2]:
prefix = 'input_strength032320'
node_points, node_connectivity, node_color = csd_functions.load_tree(prefix)

Store graph in networkx Object

In [3]:
G = nx.Graph()
G.add_edges_from(node_connectivity)

node_attributes = {node_id : {'Position' : node_points[node_id,:], 'Visited' : 0, 'Merged':[],'Inserted':[]} for node_id in list(G.nodes)}
nx.set_node_attributes(G,node_attributes)

#Make a copy for comparison
A = G.copy()



Compute multiresoluion reeb graph

In [4]:
#Number of level sets for reeb graph
interval_points = csd_functions.compute_intervals(node_points, 3)

csd_functions.graph_search(G, 0, interval_points)

In [5]:
%matplotlib qt
csd_functions.plot_graph(G)
csd_functions.plot_graph(A)


In [8]:
image_names = ['g_'+ str(i) +'.png' for i in range(1,1359)]
image_folder = 'D:/Jones_Lab/MRG_example/'
save_dir = 'D:/Jones_Lab/MRG_scan.mp4'

csd_functions.make_movie(image_folder, save_dir, image_names)

